In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold 
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
# import dummy classifier
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer


In [ ]:
df_meal = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/meal/combined_features.csv', index_col=0)
df_meal = df_meal.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])

df_play = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/play/combined_features.csv', index_col=0)
df_play = df_play.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])

In [ ]:
# Map 'no_rad' to 0 and 'rad' to 1
df_meal['label'] = df_meal['label'].map({'no_rad': 0, 'rad': 1})
df_play['label'] = df_play['label'].map({'no_rad': 0, 'rad': 1})



In [ ]:
# Reset the index
df_meal = df_meal.reset_index()

# Create the 'group' column and group by it
df_meal['group'] = df_meal['s_id'].str.split('_').str[0].astype(int)
df_grouped_meal = df_meal.drop(columns=['s_id']).groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
#df_grouped_meal['s_id'] = df_meal.groupby('group')['s_id'].first()
df_grouped_meal.index.name = 's_id'

# Reset the index
df_play = df_play.reset_index()

# Create the 'group' column and group by it
df_play['group'] = df_play['s_id'].str.split('_').str[0].astype(int)
df_grouped_play = df_play.drop(columns=['s_id']).groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
df_grouped_meal.index.name = 's_id'
df_grouped_play.index.name = 's_id'


# Reset the index of the grouped dataframes
df_grouped_meal = df_grouped_meal.reset_index()
df_grouped_play = df_grouped_play.reset_index()

# Add a new 'group' column to each DataFrame
df_grouped_meal['group'] = 'meal'
df_grouped_play['group'] = 'play'

# Concatenate the DataFrames
final_df = pd.concat([df_grouped_meal, df_grouped_play], axis=0)

final_df

In [ ]:

final_df.set_index('s_id', inplace=True)
final_df


In [ ]:
final_df

In [ ]:
final_df.set_index('group', append=True, inplace=True)


In [ ]:
# Reset the index
final_df.reset_index(inplace=True)

# Convert the first level of the index to integers
final_df.iloc[:, 0] = final_df.iloc[:, 0].astype(int)

# Set the index again
final_df.set_index(final_df.columns[:2].tolist(), inplace=True)
final_df

In [ ]:
speech = pd.read_excel('/Users/andrei-macpro/Documents/Data/classification/speech/classification.xlsx', index_col=0)

In [ ]:
# set index name to s_id
speech.index.name = 's_id'
# remove age column from speech
speech = speech.drop(columns=['age'])

In [ ]:
# Create a boolean mask where True indicates a duplicated index
mask = speech.index.duplicated(keep='first')

# Use np.where to assign 'meal' to the first occurrence and 'play' to the second
speech['group'] = np.where(mask, 'play', 'meal')

In [ ]:
speech.set_index('group', append=True, inplace=True)

In [ ]:
speech['label'] = speech['label'].map({'no_rad': 0, 'rad': 1})


In [ ]:
speech = speech.reset_index()
# set s_id as index for final_df
speech.set_index('s_id', inplace=True)
speech

In [ ]:
# drop nan rows from speech
speech = speech.dropna()

In [ ]:


final_df = final_df.reset_index()
final_df.set_index(['group', 's_id'], inplace=True)
speech = speech.reset_index()
speech.set_index(['group', 's_id'], inplace=True)
# drop label column from final_df
final_df = final_df.drop(columns=['label'])
multimodal = pd.concat([speech, final_df], axis=1)

In [ ]:
multimodal = multimodal.dropna()
multimodal

In [ ]:
multimodal = multimodal.reset_index()
multimodal.set_index(['s_id'], inplace=True)
multimodal

## Classification with all the data

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import numpy as np

# Custom transformer to apply PCA to a subset of features
class SubsetPCA(BaseEstimator, TransformerMixin):
    def __init__(self, subset_features, n_components):
        self.subset_features = subset_features
        self.n_components = n_components
        self.pca = PCA(n_components=n_components)
    
    def fit(self, data, y=None):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        self.pca.fit(data_subset)
        return self
    
    def transform(self, data):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        data_pca = self.pca.transform(data_subset)
        data_remaining = data.drop(columns=self.subset_features)
        data_pca_df = pd.DataFrame(data_pca, index=data_remaining.index)
        return pd.concat([data_remaining, data_pca_df], axis=1)

# Example data
X = multimodal[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]

subset_pca = ['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']

y = multimodal['label']
groups = multimodal.index
gkf = GroupKFold(n_splits=5)

remaining_features = [feat for feat in X.columns if feat not in subset_pca]

# Create a ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', SubsetPCA(subset_features=subset_pca, n_components=4), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])


# Define the classifiers and their parameters
classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    }),
    ('svc_linear', SVC(kernel='linear'), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf'), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
    }),
    ('et', ExtraTreesClassifier(), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'et__n_estimators': [50, 100, 200],
        'et__max_depth': [None, 5, 10, 20],
        'et__min_samples_split': [2, 5, 10]
    }),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    }),
    ('mlp', MLPClassifier(max_iter=500), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'mlp__hidden_layer_sizes': [(3,), (6,), (3, 3)],
        'mlp__activation': ['tanh', 'relu'],
        'mlp__alpha': [0.0001, 0.001, 0.01],
        'mlp__solver': ['adam', 'lbfgs']
    })
]
  
    

import pandas as pd

# Initialize a list to store the results
results = []
best_models = {}
# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled, groups_shuffled= shuffle(X, y,groups, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([ ('preprocessor', preprocessor), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled, groups=groups_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf,groups=groups_shuffled, scoring='f1_macro', n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled,groups=groups_shuffled, cv=gkf, scoring='precision_macro', n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, groups=groups_shuffled, cv=gkf, scoring='recall_macro',  n_jobs=-1)
        best_pca_components = grid_search.best_params_.get('preprocessor__pca_pipeline__n_components', None)
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'best_score': grid_search.best_score_,
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': best_pca_components
            
        })
        if name not in best_models or grid_search.best_score_ > best_models[name]['best_score']:
                best_models[name] = {
                    'best_estimator': grid_search.best_estimator_,
                    'best_params': grid_search.best_params_,
                    'best_score': grid_search.best_score_,
                    'random_state': i
                }

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)


In [ ]:
results_df.groupby('classifier').mean() 

In [ ]:
speech.iloc[:,1:-1]

In [ ]:
X

## Extracting a model just for speech 

In [ ]:

# Perform a grid search for each classifier
X = speech.iloc[:,1:-1]
y = speech['label']
groups = speech.index

# Create a GroupKFold object
gkf = GroupKFold(n_splits=5)

# Define the classifiers and their parameters
classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    }),
    ('svc_linear', SVC(kernel='linear'), {
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf'), {
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
    }),
    ('et', ExtraTreesClassifier(), {
        'et__n_estimators': [50, 100, 200],
        'et__max_depth': [None, 5, 10, 20],
        'et__min_samples_split': [2, 5, 10]
    }),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    }),
]
    
    

import pandas as pd

# Initialize a list to store the results
results = []
best_models = {}
# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled, groups_shuffled = shuffle(X, y, groups, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([('scaler', StandardScaler()),
                              (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled, groups=groups_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', groups=groups_shuffled,n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro', groups=groups_shuffled, n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro', groups=groups_shuffled, n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='accuracy',groups=groups_shuffled,  n_jobs=-1)
         # Extract the number of PCA components from the best parameters

        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'best_score': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),

        })
        # Store the best model and its parameters
        if name not in best_models or grid_search.best_score_ > best_models[name]['best_score']:
            best_models[name] = {
                'best_estimator': grid_search.best_estimator_,
                'best_params': grid_search.best_params_,
                'best_score': grid_search.best_score_
            }
# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').std()

In [ ]:
for classifier, model_info in best_models.items():
    print(f"Classifier: {classifier}")
    print(f"Best Score: {model_info['best_score']}")
    print(f"Best Parameters: {model_info['best_params']}")
    print(f"Best Estimator: {model_info['best_estimator']}\n")

In [ ]:
speech.index

In [ ]:
# Define the best parameters
best_params = {'lr__C': 1, 'lr__penalty': 'l1', 'lr__solver': 'liblinear'}

# Create the pipeline with the best parameters
best_pipeline_lr = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression(C=best_params['lr__C'], penalty=best_params['lr__penalty'], solver=best_params['lr__solver']))
])
X = speech[['Proportion speech child']]
y = speech['label']
groups = speech.index

# Create a GroupKFold object
gkf = GroupKFold(n_splits=5)
# Fit the pipeline

f1_score = cross_val_score(best_pipeline_lr, X, y, cv=gkf, scoring='f1_macro', groups=groups,n_jobs=-1)
precision_score = cross_val_score(best_pipeline_lr, X, y, cv=gkf, scoring='precision_macro', groups=groups, n_jobs=-1)
recall_score = cross_val_score(best_pipeline_lr, X, y, cv=gkf, scoring='recall_macro', groups=groups, n_jobs=-1)
accuracy_score = cross_val_score(best_pipeline_lr, X, y, cv=gkf, scoring='accuracy', groups=groups, n_jobs=-1)


In [ ]:
print(f"F1 Score: {f1_score.mean()}")
print(f"Precision: {precision_score.mean()}")
print(f"Recall: {recall_score.mean()}")
print(f"Accuracy: {accuracy_score.mean()}")

## extracting a model just for movement 


In [ ]:
final_df

In [ ]:
final_df

In [ ]:
final_df = final_df.reset_index()
final_df.set_index(['s_id'], inplace=True)

# Perform a grid search for each classifier
X = final_df[[
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]
y = final_df['label']
groups = final_df.index

# Create a GroupKFold object
gkf = GroupKFold(n_splits=5)

# Define the classifiers and their parameters
classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'pca__n_components': [2, 4],
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    }),
    ('svc_linear', SVC(kernel='linear'), {
        'pca__n_components': [2, 4],
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf'), {
        'pca__n_components': [2, 4],
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'pca__n_components': [2, 4],
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
    }),
    ('et', ExtraTreesClassifier(), {
        'pca__n_components': [2, 4],
        'et__n_estimators': [50, 100, 200],
        'et__max_depth': [None, 5, 10, 20],
        'et__min_samples_split': [2, 5, 10]
    }),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'pca__n_components': [2, 4],
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    }),
    ('mlp', MLPClassifier(max_iter=500), {
        'pca__n_components': [2, 4],
        'mlp__hidden_layer_sizes': [(3,), (6,), (3, 3)],
        'mlp__activation': ['tanh', 'relu'],
        'mlp__alpha': [0.0001, 0.001, 0.01],
        'mlp__solver': ['adam', 'lbfgs']
    })
]
    
    

import pandas as pd

# Initialize a list to store the results
results = []
best_models = {}
# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled, groups_shuffled = shuffle(X, y, groups, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([('scaler', StandardScaler()),
                             ('pca', PCA()), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled, groups=groups_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', groups=groups_shuffled,n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro', groups=groups_shuffled, n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro', groups=groups_shuffled, n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='accuracy', groups=groups_shuffled, n_jobs=-1)
         # Extract the number of PCA components from the best parameters
        best_pca_components = grid_search.best_params_.get('pca__n_components', None)
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': best_pca_components
        })
        if name not in best_models or grid_search.best_score_ > best_models[name]['best_score']:
            best_models[name] = {
                'best_estimator': grid_search.best_estimator_,
                'best_params': grid_search.best_params_,
                'best_score': grid_search.best_score_
            }

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').std()

In [ ]:
for classifier, model_info in best_models.items():
    print(f"Classifier: {classifier}")
    print(f"Best Score: {model_info['best_score']}")
    print(f"Best Parameters: {model_info['best_params']}")
    print(f"Best Estimator: {model_info['best_estimator']}\n")

In [ ]:
# Define the best parameters
best_params = {'mlp__activation': 'tanh', 'mlp__alpha': 0.01, 'mlp__hidden_layer_sizes': (50, 50), 'pca__n_components': 4}

# Create the pipeline with the best parameters
best_pipeline_mlp = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=best_params['pca__n_components'])),
    ('mlp', MLPClassifier(activation=best_params['mlp__activation'], alpha=best_params['mlp__alpha'], hidden_layer_sizes=best_params['mlp__hidden_layer_sizes']))
])

# Define the features and target
X = final_df[['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
              'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]
y = final_df['label']
groups = final_df.index

# Create a GroupKFold object
gkf = GroupKFold(n_splits=5)

# Evaluate the pipeline using cross-validation
f1_scores = cross_val_score(best_pipeline_mlp, X, y, cv=gkf, scoring='f1_macro', groups=groups, n_jobs=-1)
precision_scores = cross_val_score(best_pipeline_mlp, X, y, cv=gkf, scoring='precision_macro', groups=groups, n_jobs=-1)
recall_scores = cross_val_score(best_pipeline_mlp, X, y, cv=gkf, scoring='recall_macro', groups=groups, n_jobs=-1)
accuracy_scores = cross_val_score(best_pipeline_mlp, X, y, cv=gkf, scoring='accuracy', groups=groups, n_jobs=-1)

# Print the evaluation results
print(f"F1 Score: {f1_scores.mean()}")
print(f"Precision: {precision_scores.mean()}")
print(f"Recall: {recall_scores.mean()}")
print(f"Accuracy: {accuracy_scores.mean()}")

In [ ]:
final_df = final_df.reset_index()
final_df.set_index(['group', 's_id'], inplace=True)
final_df

In [ ]:
import pandas as pd
from scipy.stats import pearsonr
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier

# Assuming multimodal is the merged dataset with common participants
# Define features for the LR model (speech features)
X_speech = multimodal[['Proportion speech child']]
y_speech = multimodal['label']

# Define features for the MLP model (movement features)
X_movement = multimodal[['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
                         'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]
y_movement = multimodal['label']

# Define the best parameters for LR and MLP models
best_params_lr = {'lr__C': 1, 'lr__penalty': 'l1', 'lr__solver': 'liblinear'}
best_params_mlp = {'mlp__activation': 'tanh', 'mlp__alpha': 0.01, 'mlp__hidden_layer_sizes': (50, 50), 'pca__n_components': 4}

# Create the LR pipeline with the best parameters
best_pipeline_lr = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression(C=best_params_lr['lr__C'], penalty=best_params_lr['lr__penalty'], solver=best_params_lr['lr__solver']))
])

# Create the MLP pipeline with the best parameters
best_pipeline_mlp = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=best_params_mlp['pca__n_components'])),
    ('mlp', MLPClassifier(activation=best_params_mlp['mlp__activation'], alpha=best_params_mlp['mlp__alpha'], hidden_layer_sizes=best_params_mlp['mlp__hidden_layer_sizes']))
])

# Fit the pipelines to the respective datasets
best_pipeline_lr.fit(X_speech, y_speech)
best_pipeline_mlp.fit(X_movement, y_movement)

# Get predictions from the LR model
lr_predictions = best_pipeline_lr.predict_proba(X_speech)[:, 1]

# Get predictions from the MLP model
mlp_predictions = best_pipeline_mlp.predict_proba(X_movement)[:, 1]

# Compute the correlation between the predictions
correlation, p_value = pearsonr(lr_predictions, mlp_predictions)

print(f"Correlation between LR and MLP model predictions: {correlation}")
print(f"P-value: {p_value}")

In [ ]:
# Get actual predictions from the LR model
lr_predictions = best_pipeline_lr.predict(X_speech)

# Get actual predictions from the MLP model
mlp_predictions = best_pipeline_mlp.predict(X_movement)

In [ ]:
from sklearn.metrics import cohen_kappa_score

# Calculate Cohen's Kappa
kappa_score = cohen_kappa_score(lr_predictions, mlp_predictions)
print("Cohen's Kappa score:", kappa_score)

In [ ]:
from statsmodels.stats.contingency_tables import mcnemar
from sklearn.metrics import confusion_matrix

# Create a contingency table
contingency_table = confusion_matrix(lr_predictions, mlp_predictions)
result = mcnemar(contingency_table)
print("McNemar's test p-value:", result.pvalue)

In [ ]:

agreement_rate = np.mean(lr_predictions == mlp_predictions)
print("Prediction Agreement Rate:", agreement_rate)

In [ ]:
from sklearn.metrics import mutual_info_score

mi_score = mutual_info_score(lr_predictions, mlp_predictions)
print("Mutual Information score:", mi_score)

In [ ]:

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
# Define features for the LR model (speech features)
X_speech = multimodal[['Proportion speech child']]
y_speech = multimodal['label']

# Define features for the MLP model (movement features)
X_movement = multimodal[['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
                         'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]
y_movement = multimodal['label']

# Define the best parameters for LR and MLP models
best_params_lr = {'lr__C': 1, 'lr__penalty': 'l1', 'lr__solver': 'liblinear'}
best_params_mlp = {'mlp__activation': 'tanh', 'mlp__alpha': 0.01, 'mlp__hidden_layer_sizes': (50, 50), 'pca__n_components': 4}

# Create the LR pipeline with the best parameters
lr_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression(C=best_params_lr['lr__C'], penalty=best_params_lr['lr__penalty'], solver=best_params_lr['lr__solver']))
])

# Create the MLP pipeline with the best parameters
mlp_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=best_params_mlp['pca__n_components'])),
    ('mlp', MLPClassifier(activation=best_params_mlp['mlp__activation'], alpha=best_params_mlp['mlp__alpha'], hidden_layer_sizes=best_params_mlp['mlp__hidden_layer_sizes']))
])

# Combine the features for the ensemble model
X_combined = pd.concat([X_speech, X_movement], axis=1)
y_combined = multimodal['label']
groups = multimodal.index  # Assuming 'participant_id' is the column for groups

# Create a GroupKFold object
gkf = GroupKFold(n_splits=5)

# Define the base models for stacking
base_models = [
    ('lr', lr_pipeline),
    ('mlp', mlp_pipeline)
]

# Define the meta-model
meta_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Create the stacking ensemble model
stacking_ensemble = StackingClassifier(estimators=base_models, final_estimator=meta_model)

# Initialize lists to store the evaluation metrics
f1_scores = []
precision_scores = []
recall_scores = []
accuracy_scores = []

# Perform manual cross-validation
for train_index, test_index in gkf.split(X_combined, y_combined, groups):
    X_train, X_test = X_combined.iloc[train_index], X_combined.iloc[test_index]
    y_train, y_test = y_combined.iloc[train_index], y_combined.iloc[test_index]
    
    # Fit the stacking ensemble model
    stacking_ensemble.fit(X_train, y_train)
    
    # Make predictions
    y_pred = stacking_ensemble.predict(X_test)
    
    # Calculate evaluation metrics
    f1_scores.append(f1_score(y_test, y_pred, average='macro'))
    precision_scores.append(precision_score(y_test, y_pred, average='macro'))
    recall_scores.append(recall_score(y_test, y_pred, average='macro'))
    accuracy_scores.append(accuracy_score(y_test, y_pred))

# Print the evaluation results
print(f"F1 Score: {sum(f1_scores) / len(f1_scores)}")
print(f"Precision: {sum(precision_scores) / len(precision_scores)}")
print(f"Recall: {sum(recall_scores) / len(recall_scores)}")
print(f"Accuracy: {sum(accuracy_scores) / len(accuracy_scores)}")

# Stacking ensemble model

In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GroupKFold, RandomizedSearchCV, cross_val_score
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import numpy as np

# Assuming multimodal is the merged dataset with common participants
# Define features for the LR model (speech features)
X_speech = multimodal[['Proportion speech child']]
y_speech = multimodal['label']

# Define features for the MLP model (movement features)
X_movement = multimodal[['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
                         'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]
y_movement = multimodal['label']

# Define the best parameters for LR and MLP models
best_params_lr = {'lr__C': 1, 'lr__penalty': 'l1', 'lr__solver': 'liblinear'}
best_params_mlp = {'mlp__activation': 'tanh', 'mlp__alpha': 0.01, 'mlp__hidden_layer_sizes': (50, 50), 'pca__n_components': 4}

# Create the LR pipeline with the best parameters
lr_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression(C=best_params_lr['lr__C'], penalty=best_params_lr['lr__penalty'], solver=best_params_lr['lr__solver']))
])

# Create the MLP pipeline with the best parameters
mlp_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=best_params_mlp['pca__n_components'])),
    ('mlp', MLPClassifier(activation=best_params_mlp['mlp__activation'], alpha=best_params_mlp['mlp__alpha'], hidden_layer_sizes=best_params_mlp['mlp__hidden_layer_sizes']))
])

# Create additional classifiers
svc_linear_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(kernel='linear', probability=True))
])

svc_rbf_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(kernel='rbf', probability=True))
])

rf_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier())
])

et_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('et', ExtraTreesClassifier())
])

gb_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('gb', GradientBoostingClassifier())
])

# Combine the features for the ensemble model
X_combined = pd.concat([X_speech, X_movement], axis=1)
y_combined = multimodal['label']
groups = multimodal['participant_id'].values  # Assuming 'participant_id' is the column for groups

# Create a GroupKFold object
gkf = GroupKFold(n_splits=5)

# Define the base models for stacking
base_models = [
    ('lr', lr_pipeline),
    ('mlp', mlp_pipeline),
    ('svc_linear', svc_linear_pipeline),
    ('svc_rbf', svc_rbf_pipeline),
    ('rf', rf_pipeline),
    ('et', et_pipeline),
    ('gb', gb_pipeline)
]

# Define the meta-model
meta_model = RandomForestClassifier(random_state=42)

# Create the stacking ensemble model
stacking_ensemble = StackingClassifier(estimators=base_models, final_estimator=meta_model)

# Define the parameter grid for the meta-model and base models
param_distributions = {
    'final_estimator__n_estimators': [50, 100, 200],
    'final_estimator__max_depth': [None, 10, 20],
    'final_estimator__min_samples_split': [2, 5, 10],
    'lr__lr__C': [0.01, 0.1, 1, 10],
    'lr__lr__penalty': ['l1', 'l2'],
    'lr__lr__solver': ['liblinear', 'saga'],
    'mlp__mlp__activation': ['tanh', 'relu'],
    'mlp__mlp__alpha': [0.0001, 0.001, 0.01],
    'mlp__mlp__hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'svc_linear__svc__C': [0.01, 0.1, 1, 10],
    'svc_rbf__svc__C': [0.01, 0.1, 1, 10],
    'svc_rbf__svc__gamma': [0.01, 0.1, 1, 10],
    'rf__rf__n_estimators': [50, 100, 200],
    'rf__rf__max_depth': [None, 10, 20],
    'rf__rf__min_samples_split': [2, 5, 10],
    'et__et__n_estimators': [50, 100, 200],
    'et__et__max_depth': [None, 10, 20],
    'et__et__min_samples_split': [2, 5, 10],
    'gb__gb__n_estimators': [50, 100, 200],
    'gb__gb__learning_rate': [0.01, 0.1, 0.2],
    'gb__gb__max_depth': [3, 5, 10]
}

# Perform Randomized Search with cross-validation
random_search = RandomizedSearchCV(estimator=stacking_ensemble, param_distributions=param_distributions, n_iter=50, cv=gkf, scoring='f1_macro', n_jobs=-1, random_state=42)
random_search.fit(X_combined, y_combined, groups=groups)

# Get the best estimator
best_stacking_ensemble = random_search.best_estimator_

# Evaluate the best stacking ensemble using cross-validation
f1_scores = cross_val_score(best_stacking_ensemble, X_combined, y_combined, cv=gkf, scoring='f1_macro', groups=groups, n_jobs=-1)
precision_scores = cross_val_score(best_stacking_ensemble, X_combined, y_combined, cv=gkf, scoring='precision_macro', groups=groups, n_jobs=-1)
recall_scores = cross_val_score(best_stacking_ensemble, X_combined, y_combined, cv=gkf, scoring='recall_macro', groups=groups, n_jobs=-1)
accuracy_scores = cross_val_score(best_stacking_ensemble, X_combined, y_combined, cv=gkf, scoring='accuracy', groups=groups, n_jobs=-1)

# Print the evaluation results
print(f"Best Parameters: {random_search.best_params_}")
print(f"F1 Score: {f1_scores.mean()}")
print(f"Precision: {precision_scores.mean()}")
print(f"Recall: {recall_scores.mean()}")
print(f"Accuracy: {accuracy_scores.mean()}")

# Voting Classifier

In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GroupKFold, cross_val_score
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

# Assuming multimodal is the merged dataset with common participants
# Define features for the LR model (speech features)
X_speech = multimodal[['Proportion speech child']]
y_speech = multimodal['label']

# Define features for the MLP model (movement features)
X_movement = multimodal[['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
                         'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]
y_movement = multimodal['label']

# Define the best parameters for LR and MLP models
best_params_lr = {'lr__C': 1, 'lr__penalty': 'l1', 'lr__solver': 'liblinear'}
best_params_mlp = {'mlp__activation': 'tanh', 'mlp__alpha': 0.01, 'mlp__hidden_layer_sizes': (50, 50), 'pca__n_components': 4}

# Create the LR pipeline with the best parameters
lr_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression(C=best_params_lr['lr__C'], penalty=best_params_lr['lr__penalty'], solver=best_params_lr['lr__solver']))
])

# Create the MLP pipeline with the best parameters
mlp_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=best_params_mlp['pca__n_components'])),
    ('mlp', MLPClassifier(activation=best_params_mlp['mlp__activation'], alpha=best_params_mlp['mlp__alpha'], hidden_layer_sizes=best_params_mlp['mlp__hidden_layer_sizes']))
])



# Combine the features for the ensemble model
X_combined = pd.concat([X_speech, X_movement], axis=1)
y_combined = multimodal['label']
groups = multimodal.index  # Assuming 'participant_id' is the column for groups

# Create a GroupKFold object
gkf = GroupKFold(n_splits=5)

# Define the base models for voting
base_models = [
    ('lr', lr_pipeline),
    ('mlp', mlp_pipeline)
]

# Create the voting classifier
voting_ensemble = VotingClassifier(estimators=base_models, voting='soft')

# Evaluate the voting ensemble using cross-validation
f1_scores = cross_val_score(voting_ensemble, X_combined, y_combined, cv=gkf, scoring='f1_macro', groups=groups, n_jobs=-1)
precision_scores = cross_val_score(voting_ensemble, X_combined, y_combined, cv=gkf, scoring='precision_macro', groups=groups, n_jobs=-1)
recall_scores = cross_val_score(voting_ensemble, X_combined, y_combined, cv=gkf, scoring='recall_macro', groups=groups, n_jobs=-1)
accuracy_scores = cross_val_score(voting_ensemble, X_combined, y_combined, cv=gkf, scoring='accuracy', groups=groups, n_jobs=-1)

# Print the evaluation results
print(f"F1 Score: {f1_scores.mean()}")
print(f"Precision: {precision_scores.mean()}")
print(f"Recall: {recall_scores.mean()}")
print(f"Accuracy: {accuracy_scores.mean()}")

# Weighted Averaging

In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GroupKFold, cross_val_predict
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import numpy as np

# Assuming multimodal is the merged dataset with common participants
# Define features for the LR model (speech features)
X_speech = multimodal[['Proportion speech child']]
y_speech = multimodal['label']

# Define features for the MLP model (movement features)
X_movement = multimodal[['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
                         'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]
y_movement = multimodal['label']

# Define the best parameters for LR and MLP models
best_params_lr = {'lr__C': 1, 'lr__penalty': 'l1', 'lr__solver': 'liblinear'}
best_params_mlp = {'mlp__activation': 'tanh', 'mlp__alpha': 0.01, 'mlp__hidden_layer_sizes': (50, 50), 'pca__n_components': 4}

# Create the LR pipeline with the best parameters
lr_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression(C=best_params_lr['lr__C'], penalty=best_params_lr['lr__penalty'], solver=best_params_lr['lr__solver']))
])

# Create the MLP pipeline with the best parameters
mlp_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=best_params_mlp['pca__n_components'])),
    ('mlp', MLPClassifier(activation=best_params_mlp['mlp__activation'], alpha=best_params_mlp['mlp__alpha'], hidden_layer_sizes=best_params_mlp['mlp__hidden_layer_sizes']))
])

# Combine the features for the ensemble model
X_combined = pd.concat([X_speech, X_movement], axis=1)
y_combined = multimodal['label']
groups = multimodal.index  # Assuming 'participant_id' is the column for groups

# Create a GroupKFold object
gkf = GroupKFold(n_splits=5)

# Define the base models
base_models = [
    ('lr', lr_pipeline),
    ('mlp', mlp_pipeline)
]

# Define weights for the base models
weights = [0.7, 0.3]  # Adjust these weights based on model performance

# Initialize lists to store the evaluation metrics
f1_scores = []
precision_scores = []
recall_scores = []
accuracy_scores = []

# Perform manual cross-validation
for train_index, test_index in gkf.split(X_combined, y_combined, groups):
    X_train, X_test = X_combined.iloc[train_index], X_combined.iloc[test_index]
    y_train, y_test = y_combined.iloc[train_index], y_combined.iloc[test_index]
    
    # Fit the base models
    for name, model in base_models:
        model.fit(X_train, y_train)
    
    # Get the predicted probabilities from each base model
    lr_probs = lr_pipeline.predict_proba(X_test)[:, 1]
    mlp_probs = mlp_pipeline.predict_proba(X_test)[:, 1]
    
    # Combine the predicted probabilities using weighted averaging
    combined_probs = (weights[0] * lr_probs) + (weights[1] * mlp_probs)
    
    # Convert combined probabilities to class predictions
    combined_preds = np.where(combined_probs > 0.5, 1, 0)
    
    # Calculate evaluation metrics
    f1_scores.append(f1_score(y_test, combined_preds, average='macro'))
    precision_scores.append(precision_score(y_test, combined_preds, average='macro'))
    recall_scores.append(recall_score(y_test, combined_preds, average='macro'))
    accuracy_scores.append(accuracy_score(y_test, combined_preds))

# Print the evaluation results
print(f"F1 Score: {np.mean(f1_scores)}")
print(f"Precision: {np.mean(precision_scores)}")
print(f"Recall: {np.mean(recall_scores)}")
print(f"Accuracy: {np.mean(accuracy_scores)}")

# Use base models predictions as features

In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GroupKFold, cross_val_predict
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import numpy as np

# Assuming multimodal is the merged dataset with common participants
# Define features for the LR model (speech features)
X_speech = multimodal[['Proportion speech child']]
y_speech = multimodal['label']

# Define features for the MLP model (movement features)
X_movement = multimodal[['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
                         'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]
y_movement = multimodal['label']

# Combine the features for the ensemble model
X_combined = pd.concat([X_speech, X_movement], axis=1)
y_combined = multimodal['label']
groups = multimodal.index  # Assuming 'participant_id' is the column for groups

# Create a GroupKFold object
gkf = GroupKFold(n_splits=5)

# Define the base models
base_models = [
    ('lr', Pipeline([
        ('scaler', StandardScaler()),
        ('lr', LogisticRegression(C=1, penalty='l1', solver='liblinear'))
    ])),
    ('mlp', Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=4)),
        ('mlp', MLPClassifier(activation='tanh', alpha=0.01, hidden_layer_sizes=(50, 50)))
    ]))
]

# Initialize an empty DataFrame to store the new features
new_features = pd.DataFrame(index=X_combined.index)

# Generate out-of-fold predictions for each base model
for name, model in base_models:
    oof_preds = cross_val_predict(model, X_combined, y_combined, cv=gkf, method='predict_proba', groups=groups, n_jobs=-1)[:, 1]
    new_features[name] = oof_preds

# Combine the new features with the original features
X_meta = pd.concat([X_combined, new_features], axis=1)

# Define the meta-model
meta_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Evaluate the meta-model using cross-validation
f1_scores = cross_val_score(meta_model, X_meta, y_combined, cv=gkf, scoring='f1_macro', groups=groups, n_jobs=-1)
precision_scores = cross_val_score(meta_model, X_meta, y_combined, cv=gkf, scoring='precision_macro', groups=groups, n_jobs=-1)
recall_scores = cross_val_score(meta_model, X_meta, y_combined, cv=gkf, scoring='recall_macro', groups=groups, n_jobs=-1)
accuracy_scores = cross_val_score(meta_model, X_meta, y_combined, cv=gkf, scoring='accuracy', groups=groups, n_jobs=-1)

# Print the evaluation results
print(f"F1 Score: {f1_scores.mean()}")
print(f"Precision: {precision_scores.mean()}")
print(f"Recall: {recall_scores.mean()}")
print(f"Accuracy: {accuracy_scores.mean()}")

In [ ]:
X_meta